In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
pd.get_option("display.max_columns")

20

## Load Data

In [2]:
path = '../data/raw'
articles = pd.read_csv(os.path.join(path,'articles.csv'),dtype={'article_id':str})
customers = pd.read_csv(os.path.join(path,'customers.csv'))
transactions = pd.read_csv(os.path.join(path,'transactions_train.csv'),dtype={'article_id':str})
articles.head()

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,0108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,0108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,0108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,0110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,0110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."


### Reduce Memory Usage

The customer_id is a length 64 string which uses 64 bytes.<br> The code will coverts the column to int64 which only takes 8 bytes! Also the mapping is 1:1 

In [4]:
id_to_index_dict = dict(zip(customers["customer_id"], customers.index))
index_to_id_dict = dict(zip(customers.index, customers["customer_id"]))

# for memory efficiency
customers["customer_id"] = customers["customer_id"].map(id_to_index_dict)
transactions["customer_id"] = transactions["customer_id"].map(id_to_index_dict)
customers.head()

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,0,NaN,NaN,ACTIVE,NONE,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,1,NaN,NaN,ACTIVE,NONE,25.0,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
2,2,NaN,NaN,ACTIVE,NONE,24.0,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
3,3,NaN,NaN,ACTIVE,NONE,54.0,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...
4,4,1.0,1.0,ACTIVE,Regularly,52.0,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...


In [5]:
### Convert other columns
transactions['price'] = transactions['price'].astype('float32')
transactions['sales_channel_id'] = transactions['sales_channel_id'].astype('int8')
transactions['t_dat'] = pd.to_datetime(transactions['t_dat'],format = '%Y-%m-%d')
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31788324 entries, 0 to 31788323
Data columns (total 5 columns):
 #   Column            Dtype         
---  ------            -----         
 0   t_dat             datetime64[ns]
 1   customer_id       int64         
 2   article_id        object        
 3   price             float32       
 4   sales_channel_id  int8          
dtypes: datetime64[ns](1), float32(1), int64(1), int8(1), object(1)
memory usage: 879.2+ MB


## Data Quality 

To-Do (Data Quality)
- Define Column's Definition
- Check Null Columns
- Identify Categorical variables and its quality
- Plot Histogram for numericle variables (Check Normality)
- Find Outlier

### Check Missing values

#### Articles

In [ ]:
articles.info()

# Check Missing Value in Each Column
count_null = pd.DataFrame(articles.isnull().sum(),columns=['null_items'])
count_null['null_ratio'] = count_null['null_items']/len(articles)
count_null[count_null.null_items > 0].T

#### Customers

In [6]:
customers.info()

# Check Missing Value in Each Column
count_null = pd.DataFrame(customers.isnull().sum(),columns=['null_items'])
count_null['null_ratio'] = count_null['null_items']/len(customers)
count_null[count_null.null_items > 0]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1371980 entries, 0 to 1371979
Data columns (total 7 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   customer_id             1371980 non-null  int64  
 1   FN                      476930 non-null   float64
 2   Active                  464404 non-null   float64
 3   club_member_status      1365918 non-null  object 
 4   fashion_news_frequency  1355971 non-null  object 
 5   age                     1356119 non-null  float64
 6   postal_code             1371980 non-null  object 
dtypes: float64(3), int64(1), object(3)
memory usage: 73.3+ MB


,null_items,null_ratio
FN,895050,0.652378
Active,907576,0.661508
club_member_status,6062,0.004418
fashion_news_frequency,16009,0.011669
age,15861,0.011561


#### Transactions

In [7]:
transactions.info()

# Check Missing Value in Each Column
count_null = pd.DataFrame(transactions.isnull().sum(),columns=['null_items'])
count_null['null_ratio'] = count_null['null_items']/len(transactions)
count_null[count_null.null_items > 0].T

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31788324 entries, 0 to 31788323
Data columns (total 5 columns):
 #   Column            Dtype         
---  ------            -----         
 0   t_dat             datetime64[ns]
 1   customer_id       int64         
 2   article_id        object        
 3   price             float32       
 4   sales_channel_id  int8          
dtypes: datetime64[ns](1), float32(1), int64(1), int8(1), object(1)
memory usage: 879.2+ MB


""
null_items
null_ratio


### Explore Missing Values

Only "customer.csv" has missing value problems in the following columns:
- [FN]
- [Active]
- [club_member_status]
- [fashion_news_frequency]
- [age]

In [ ]:
print('row percentage of missing value is {} %'.format(len(customers[customers.isnull().any(axis=1)])*100/customers.shape[0]))

# Check Missing Value in Each Column
count_null = pd.DataFrame(customers.isnull().sum(),columns=['null_items'])
count_null['null_ratio'] = count_null['null_items']/len(customers)
count_null[count_null.null_items > 0].T

#### FN

In [ ]:
pd.DataFrame(customers[['FN']].value_counts(dropna=False))

#### Active

In [ ]:
pd.DataFrame(customers[['Active']].value_counts(dropna=False))

#### club_member_status

In [ ]:
pd.DataFrame(customers[['club_member_status']].value_counts(dropna=False))

#### fashion_news_frequency

In [ ]:
pd.DataFrame(customers[['fashion_news_frequency']].value_counts(dropna=False))

In [ ]:
pd.DataFrame(customers[['Active','club_member_status']].value_counts(dropna=False))

#### age

In [ ]:
customers['age'].hist()

In [ ]:
customers[['age']].describe().astype(str).T

In [ ]:
temp = customers.copy()
temp['age_bin'] = pd.cut(temp['age'],bins=[min(temp['age']),35,60,80,max(temp['age'])])

pd.DataFrame(temp['age_bin'].value_counts(dropna=False)).reset_index()

#### Cremer's V Test

In [ ]:
for i in [ i for i in customers.columns if customers[i].isnull().any()]:
    temp[i+'_m'] = np.where(temp[i].isnull(),True,False)
    
# Convert boolean to categary for Cremer's V Calculation    
temp = temp.apply(lambda x: x.astype("category") if x.dtype == "bool" else x)

temp.head()

In [ ]:
temp[['FN','Active']]  = temp[['FN','Active']].fillna(0)
temp[['FN','Active','club_member_status','fashion_news_frequency']] = temp[['FN','Active','club_member_status','fashion_news_frequency']].astype('category')

In [ ]:
import association_metrics as am
# Initialize a Cramer V object
cramersv = am.CramersV(temp)

In [ ]:
cramersv.fit()[['FN_m','Active_m','club_member_status_m','fashion_news_frequency_m','age_m']][:5]


In [ ]:
customers.fillna(0).groupby(['FN','Active','fashion_news_frequency']).count()

In [ ]:
temp.groupby(['FN_m','fashion_news_frequency']).count()

In [ ]:
temp.groupby(['Active_m','fashion_news_frequency']).count()

In [ ]:
temp.groupby(['age_m','club_member_status']).count()

### Detect Outlier

In [ ]:
fig, (ax0, ax1) = plt.subplots(nrows=1, ncols=2, figsize=(8, 4))

ax0.boxplot(transactions['price'],labels=['Price'])
ax1.hist(transactions['price'][transactions['price']<=0.3],bins=30)

plt.show()
pd.DataFrame(transactions['price'].describe()).astype(float).T

In [ ]:
fig, (ax0, ax1) = plt.subplots(nrows=1, ncols=2, figsize=(8, 4))

ax0.boxplot(customers['age'],labels=['age'])
ax1.hist(customers['age'],bins=30)

plt.show()
pd.DataFrame(customers['age'].describe(percentiles=[0.1,0.25,0.5,0.75,0.9])).T

### Merge Problem

#### Unsold articles

In [ ]:
temp = transactions.merge(articles,on='article_id',how='outer',indicator=True)
print('Total unsold articles: ',len(temp[temp._merge == 'right_only']))
temp[temp._merge == 'right_only'].head(2)

#### Cold Start customers (Zero transaction)

In [ ]:
zero_tran_cus = customers.merge(transactions,on='customer_id',how='outer',indicator=True)
print('Total zero-transaction customers: ',len(zero_tran_cus[zero_tran_cus._merge == 'left_only']))
zero_tran_cus = zero_tran_cus[zero_tran_cus._merge == 'left_only']
zero_tran_cus.head()

<b>Note: </b>I suggest that we should remove these clients from our modelling process because we don't have their detail much.

## Data Transformation
Only customer.csv that should be fixed due to its missing data.

#### Remove zero-transaction customers.


In [ ]:
# print('Total Records (Before): ',len(customers))
# customers = customers[~customers['customer_id'].isin(zero_tran_cus['customer_id'])].reset_index(drop=True)
# print('Total Records (After): ',len(customers))


#### Impute null values for each column.

In [8]:
customers[['FN','Active']] = customers[['FN','Active']].fillna(0.0)
customers[['age']] = customers[['age']].fillna(customers.age.median())
customers[['club_member_status']] = customers[['club_member_status']].fillna('OTHER')
customers[['fashion_news_frequency']] = customers[['fashion_news_frequency']].fillna('NONE')
customers['fashion_news_frequency'] = customers['fashion_news_frequency'].apply(lambda x: 'NONE' if x == 'None' else x)
customers.head()

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,0,0.0,0.0,ACTIVE,NONE,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,1,0.0,0.0,ACTIVE,NONE,25.0,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
2,2,0.0,0.0,ACTIVE,NONE,24.0,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
3,3,0.0,0.0,ACTIVE,NONE,54.0,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...
4,4,1.0,1.0,ACTIVE,Regularly,52.0,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...


#### Join table

In [9]:
temp = transactions.merge(customers,how='left',on='customer_id')
temp.drop(columns=['postal_code'],inplace=True)
temp = temp.merge(articles,how='left',on='article_id')
# for i in temp.select_dtypes(include=['float64']).columns:
#     temp[i] = temp[i].astype('float32')
temp.head()

,t_dat,customer_id,article_id,price,sales_channel_id,FN,Active,club_member_status,fashion_news_frequency,age,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,2018-09-20,2,0663713001,0.050831,2,0.0,0.0,ACTIVE,NONE,24.0,...,Expressive Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Lace push-up body with underwired, moulded, pa..."
1,2018-09-20,2,0541518023,0.030492,2,0.0,0.0,ACTIVE,NONE,24.0,...,Casual Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Lace push-up bras with underwired, moulded, pa..."
2,2018-09-20,7,0505221004,0.015237,2,1.0,1.0,ACTIVE,Regularly,32.0,...,Tops Knitwear DS,D,Divided,2,Divided,58,Divided Selected,1003,Knitwear,Jumper in rib-knit cotton with hard-worn detai...
3,2018-09-20,7,0685687003,0.016932,2,1.0,1.0,ACTIVE,Regularly,32.0,...,Campaigns,A,Ladieswear,1,Ladieswear,15,Womens Everyday Collection,1023,Special Offers,V-neck knitted jumper with long sleeves and ri...
4,2018-09-20,7,0685687004,0.016932,2,1.0,1.0,ACTIVE,Regularly,32.0,...,Campaigns,A,Ladieswear,1,Ladieswear,15,Womens Everyday Collection,1023,Special Offers,V-neck knitted jumper with long sleeves and ri...


## Save processed data

In [10]:
import pickle
# mapping index
path = '../data/processed'

# Save index for customer_id
with open(os.path.join(path,'cusId_to_index.pkl'), 'wb') as f:
    pickle.dump(id_to_index_dict, f)
    
with open(os.path.join(path,'index_to_cusId.pkl'), 'wb') as f:
    pickle.dump(index_to_id_dict, f)    

articles.to_pickle(os.path.join(path,'articles.pkl'))
customers.to_pickle(os.path.join(path,'customers.pkl'))
transactions.to_pickle(os.path.join(path,'transactions.pkl'))
temp.to_pickle(os.path.join(path,'merged_data.pkl'))